## Add support for equality constraints to Newton's Method

### Find argmin[f(x)] s.t. c(x) = 0 by using Newton's Method

In [1]:
using LinearAlgebra
using ForwardDiff

In [2]:
function newton(f, x0, c;
                grad=x->ForwardDiff.gradient(f,x),
                hess=x->ForwardDiff.hessian(f,x),
                ftol = 1e-6,
                xtol = 1e-4, gtol=1-6, maxiter = 1000, 
                xrange=[-2., 3.],
                yrange=[-2.,6.], animate=true)
  fold = f(x0)
  xold = x0
  xchange = Inf
  fchange = Inf
  iter = 0
  stuck = 0

  g = grad(xold)
  
  # Here I apply the constraint equation
  all(c(x0) .≈ 0) || error("interiorpoint requires a starting value that strictly satisfies all constraints")
    
  while(iter < maxiter && ((xchange>xtol) || (fchange>ftol) || (stuck>0)
                           || norm(g)>gtol) )
    g = grad(xold)
    H = hess(xold)
    Δx = - inv(H)*g
    x = xold + Δx
    fnew = f(x)
    step = 1.0
    acceptedstep = false
        
    while (step>xtol)
      step /= 1.618
      x = xold + Δx*step
      if (all(c(x0) .≈ 0))   # Here I apply the constraint equation
        acceptedstep = true
        break
      end
    end
    
    # If constraint equation has never been satisfied, the algorithm stops
    if !acceptedstep                      
      stuck = 1
      break
    end
    
    # Otherwise, the algorithm proceeds as usual
    fnew = f(x)
        
    if (fnew>=fold)
      stuck += 1
      if (stuck>=10)
        break
      end
    else
      stuck = 0
      xold = x
      fold = fnew
    end
        
    xchange = norm(x-xold)
    fchange = abs(fnew-fold)
    iter += 1
  end
    
  if (iter >= maxiter)
    info = "Maximum iterations reached"
  elseif (stuck > 0)
    info = "Failed to improve for " * string(stuck) * " iterations."
  else
    info = "Convergence."
  end
  return(fold, xold, iter, info) 
end

newton (generic function with 1 method)

In [3]:
function banana(a,b)
  x->(a-x[1])^2+b*(x[2]-x[1]^2)^2
end
f = banana(1.0,1.0)

x0 = [3.0, 3.0];

In [4]:
function constraint(x)
  [x[1] - x[2]]
end

constraint (generic function with 1 method)

In [5]:
result = newton(f, x0, constraint)

(4.930380657631324e-32, [1.0000000000000002, 1.0000000000000004], 52, "Failed to improve for 10 iterations.")

#### Adding the equality constraints leads to more number of iterations in the algorithm